In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
import pytorch_lightning as pl

import pandas as pd
import numpy as np
import joblib
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn import model_selection

# break

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, input_cols, cond_cols, target):
        self.inputs = torch.FloatTensor(data[input_cols].to_numpy())
        self.conds = torch.FloatTensor(dadta[cond_cols].to_numpy())
        self.target = torch.FloatTensor(data[target].to_numpy())
    
    def __getitem__(self, i):
        return self.inputs[i], self.conds[i], self.target[i]

    def __len__(self):
        return len(self.target)

In [40]:
class CTRPDataModule(pl.LightningDataModule):

    def __init__(self, train, val, fold, input_cols, cond_cols, target, batch_size=32):
        super().__init__()
        self.train = train
        self.val = val
        self.fold = fold
        self.input_cols = input_cols
        self.cond_cols = cond_cols
        self.target = target
        self.batch_size = batch_size

    # When doing distributed training, Datamodules have two optional arguments for
    # granular control over download/prepare/splitting data:
    def prepare_data(self):
        pass

    # OPTIONAL, called for every GPU/machine (assigning state is OK)
    def setup(self, stage):
        # transformations
        self.scaler = StandardScaler()
        self.train[self.input_cols] = self.scaler.fit_transform(self.train[self.input_cols])
        self.val[self.input_cols] = self.scaler.transform(self.val[self.input_cols])
        
        if stage == 'fit':
            self.train_dataset = Dataset(self.train, self.input_cols, self.cond_cols, self.target)
            self.val_dataset = Dataset(self.val, self.input_cols, self.cond_cols, self.target)
            return self.train_dataset, self.val_dataset
        if stage == 'test':
            self.test_dataset = Dataset(self.test, self.input_cols, self.cond_cols, self.target)
            return self.test_dataset

    # return the dataloader for each split
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=8, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [ ]:
data_path = Path("../../film-gex-data/processed/")
input_cols = joblib.load(data_path.joinpath("input_cols.pkl"))
cond_cols = joblib.load(data_path.joinpath("cond_cols.pkl"))
data = pd.read_pickle(data_path.joinpath("train_sub.pkl.gz"))

In [26]:
n_splits = 5
target = "cpd_avg_pv"
group = "stripped_cell_line_name"

In [43]:
gkf = model_selection.GroupKFold(n_splits=n_splits, random_state=42)

for fold, (train_idx, val_idx) in enumerate(gkf.split(X=data, y=data[target].to_numpy(), groups=data[group].to_numpy())):
    train = data.iloc[train_idx]
    val = data.iloc[val_idx]
    
    dm = CTRPDataModule(train,
                        val,
                        fold,
                        input_cols,
                        cond_cols,
                        target=target,
                        batch_size=128)
    break

TypeError: split() got an unexpected keyword argument 'random_state'